In [1]:
import ollama
import requests
import json

In [2]:
model = "llama3.2"

#### Task 1: Interact with deployed LLM via python 


**Objective:**

Explore different techniques to interact with the deployed LLM.

**Task Description:**

1. Use Request libaray (HTTP Client) and send a POST request to interact with the LLM: [How To](https://requests.readthedocs.io/en/latest/user/quickstart/#make-a-request)

In [4]:
# Simple HTTP Request via requests

# Define the URL of the deployed LLM ( this port is forwarded from the docker container to the host system)
url = "http://localhost:11434/api/generate"

# Define the prompt
body = {
    "model": model,
    "prompt": "Describe Generative AI in two sentences."
}

# ADD HERE YOUR CODE
# Send the POST request
response = requests.post(url=url, json=body)

# Check if the request was successful
if response.status_code == 200:
    # Process the response
    response_text = response.text

    # Convert each line to json
    response_lines = response_text.splitlines()
    response_json = [json.loads(line) for line in response_lines]
    for line in response_json:
        # Print the response. No line break
        print(line["response"], end="")
else:
    print("Error:", response.status_code)


Generative AI refers to a type of artificial intelligence that creates new, original content, such as images, videos, text, or music, based on patterns and styles learned from existing data, allowing it to generate novel outputs that are often indistinguishable from human-created works. This technology has the potential to revolutionize various fields, including art, design, writing, and entertainment, by automating the creative process and enabling new forms of artistic expression.

**Task Description:**

2. Use Ollama python library to interact with the LLM: [How To](https://pypi.org/project/ollama/)

- First use method ``ollama.chat(...)``
- First use method ``ollama.chat(...)`` with ``stream=True``

In [10]:
# API Call via ollama

# ADD HERE YOUR CODE

response = ollama.chat(
  model='llama3.2',
  messages=[{
    'role': 'user',
    'content': 'Moin Meister'
  }],
  )


print(response["message"]["content"])

Hallo! Wie kann ich dir heute helfen?


In [11]:
# Streaming API Call via ollama

# Response streaming can be enabled by setting stream=True, 
# modifying function calls to return a Python generator where each part is an object in the stream.

# ADD HERE YOUR CODE

stream = ollama.chat(
  model='llama3.2',
  messages=[{
    'role': 'user',
    'content': 'Moin Meister'
  }],
  stream=True
  )

for chunk in stream:
  print(chunk["message"]["content"], end="", flush=True)

Moin! Wie kann ich Ihnen heute helfen?

#### Task 2: Experimenting with Prompt Techniques

**Objective:**

Objective: Explore different prompt techniques (Zero Shot, One Shot, and Few Shot) by sending different types of prompts to the LLM.

![image](https://miro.medium.com/v2/resize:fit:1400/format:webp/1*QSpK--jqPiUU_OHuZvtUWA.png)

**Task Description:**

1. Create three prompts for a sentiment analysis task: a Zero Shot prompt, a One Shot prompt, and a Few Shot prompt. Use the examples from the table above.
2. Send these prompts to the LLM and observe the differences in the responses.
3. Compare and discuss the responses.

In [12]:
# ADD HERE YOUR PROMPTS

zero_shot_prompt = "Klassifiziere die Aussage: Ich liebe Informatik! Sentiment: "

one_shot_prompt = "Klassifiziere die Aussage: Ich liebe Informatik! Sentiment: Positiv. Klassifiziere die Aussage: Ich hasse Mathe 2. Sentiment:"

few_shot_prompt = "Klassifiziere die Aussage: Ich liebe Informatik! Sentiment: Positiv. Klassifiziere die Aussage: Ich hasse Mathe 2. Sentiment: Negativ. Klassifiziere die Aussage: Ich geh nur manchmal in Statistik. Sentiment:"

# Stream the responses and print them
for idx, prompt in enumerate([zero_shot_prompt, one_shot_prompt, few_shot_prompt]):
    prompt_type = ["Zero-Shot", "One-Shot", "Few-Shot"][idx]
    print(f"\n--- {prompt_type} Prompt ---\n")
    print(f"User Prompt:\n{prompt}\n")
    
    stream = ollama.chat(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        stream=True,
    )
    
    print("Model Output:")
    for chunk in stream:
        print(chunk["message"]["content"], end="", flush=True)
    print("\n-----------------------------\n")



--- Zero-Shot Prompt ---

User Prompt:
Klassifiziere die Aussage: Ich liebe Informatik! Sentiment: 

Model Output:
Die Klassifikation der Aussage "Ich liebe Informatik!" in Bezug auf das Gefühl oder den Sentiment kann wie folgt erfolgen:

- **Objektive Analyse**: Die Aussage ist ein deklaratives Statement über eine persönliche Vorliebe.
- **Subjektive Interpretation**: Die Aussage zeigt ein positives Gefühl gegenüber der Informatik, da die Wörter "Ich liebe" positiv konnotiert sind. Dies deutet darauf hin, dass die Person die Informatik sehr schätzt und möglicherweise eine Leidenschaft für sie hat.

Insgesamt kann man sagen, dass die Aussage ein positives Gefühl ausdrückt und somit einen **positiven** Sentiment enthält.
-----------------------------


--- One-Shot Prompt ---

User Prompt:
Klassifiziere die Aussage: Ich liebe Informatik! Sentiment: Positiv. Klassifiziere die Aussage: Ich hasse Mathe 2. Sentiment:

Model Output:
Die Klassifikation der Aussagen:

1. "Ich liebe Informatik

#### Task 3: Prompt Refinement and Optimization

**Objective:** 

Refine a prompt to improve the clarity and quality of the LLM's response.

**Task Description:**

- Start with a basic prompt asking the LLM to summarize a paragraph.
- Refine the prompt by adding specific instructions to improve the summary's quality. (Example: define how long the summary should be, define on which to focus in the summary)

In [13]:
# Original prompt
original_prompt = "Summarize the following paragraph: Generative AI is a field of artificial intelligence focused on creating new content based on patterns learned from existing data. It has applications in text, image, and music generation, and is increasingly being used in creative industries."

# ADD HERE YOUR PROMPT
refined_prompt = "Summarize the following paragraph in a single simple sentence, focusing on the applications of AI: Generative AI is a field of artificial intelligence focused on creating new content based on patterns learned from existing data. It has applications in text, image, and music generation, and is increasingly being used in creative industries."

# Stream the responses and print them
for idx, prompt in enumerate([original_prompt, refined_prompt]):
    prompt_type = ["Original Prompt", "Refined Prompt"][idx]
    print(f"\n--- {prompt_type} ---\n")
    print(f"User Prompt:\n{prompt}\n")
    
    stream = ollama.chat(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        stream=True,
    )
    
    print("Model Output:")
    for chunk in stream:
        print(chunk["message"]["content"], end="", flush=True)
    print("\n-----------------------------\n")



--- Original Prompt ---

User Prompt:
Summarize the following paragraph: Generative AI is a field of artificial intelligence focused on creating new content based on patterns learned from existing data. It has applications in text, image, and music generation, and is increasingly being used in creative industries.

Model Output:
Generative AI is a type of artificial intelligence that creates new content by analyzing and mimicking patterns found in existing data, with applications in text, image, and music generation.
-----------------------------


--- Refined Prompt ---

User Prompt:
Summarize the following paragraph in a single simple sentence, focusing on the applications of AI: Generative AI is a field of artificial intelligence focused on creating new content based on patterns learned from existing data. It has applications in text, image, and music generation, and is increasingly being used in creative industries.

Model Output:
Artificial Intelligence (AI) generates new content

#### [Optional] Task 4: Structured Prompting with Roles (Pirate Theme)

**Objective:**

Learn how to use structured prompts that combine role assignment, clear instructions, and examples to improve the output of language models. In this task, you will guide the AI to respond as a pirate who is also an expert in machine learning.

**Instructions:**

- Role Assignment: In your prompt, specify the role of the AI as a Machine Learning Expert who speaks like a pirate.

- Instruction: Clearly state what you want the AI to explain or discuss in pirate language.

- Examples: Provide examples to guide the AI in using pirate lingo while explaining technical concepts.

In [14]:
# Combined Techniques Prompt with Pirate Theme

structured_prompt = "Zugewiesene Rolle: Du bist ein KI-Experte der wie ein Pirat spricht. Anweisung: Wie kann eine KI aus nur einem Satz eine detailierte Erklärung generieren. Beispiele: "

# Stream the response and print it
print("=== User Prompt ===")
print(structured_prompt)

stream = ollama.chat(
    model=model,
    messages=[{"role": "user", "content": structured_prompt}],
    stream=True,
)

print("\n=== Model Output ===")
for chunk in stream:
    print(chunk["message"]["content"], end="", flush=True)
print("\n")


=== User Prompt ===
Zugewiesene Rolle: Du bist ein KI-Experte der wie ein Pirat spricht. Anweisung: Wie kann eine KI aus nur einem Satz eine detailierte Erklärung generieren. Beispiele: 

=== Model Output ===
Meine Schätze! Eine KI, die aus nur einem Satz eine detailierte Erklärung generieren kann, ist wie ein wunderbarer Goldschmied, der aus einem einzigen Stück Metall eine Schatzkiste voller Geheimnisse und Wissens formt.

Hier sind einige Geheimnisse, die ich dir verrate, meine Herren:

1. **Naturmaschinelles Lernen**: Die KI nutzt ihre Fähigkeit, Muster in Daten zu erkennen, um sich selbst zu verbessern. Es ist wie ein wunderbares Schiff, das durch die Wellen des Wetters navigiert und sich langsam, aber sicher an die Geheimnisse der Daten aneignet.
2. **Textanalyse**: Die KI verwendet ihre Fähigkeit, Texte zu analysieren, um die Bedeutung von Wörtern und Sätzen zu erkennen. Es ist wie ein wunderbarer Spion, der die geheimen Nachrichten in den Staub des alten Papiers herausfindet.
3